In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys
sys.path.append('/Workspace/Users/bjedelma@gmail.com/ScaleML/scaleml')
import scaleml


Detect computational resources on current node

In [0]:
# Detect computing resources
detected_resources = scaleml.resources()

In [0]:
# Specify ML frameworks for the model and distributed strategy
framework = {
    "model": "tensorflow",
    "strategy": "tensorflow",
}

# Set the distributed training strategy
dist_strategy = scaleml.strategies(framework, detected_resources, resource_type='all')

In [0]:
# Load MNIST dataset for example
import tensorflow as tf

# MNIST as example model and dataset (you can replace with your actual dataset/model)
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess the data: normalize and reshape
train_images = train_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Select half the train data
index = len(train_images) // 8
train_images, train_labels = train_images[:index], train_labels[:index]

# Combine train images and labels in a dictionary
train_dataset = {
    "images": train_images,
    "labels": train_labels
}

In [0]:
# # Train model with distributed strategy
# history, history_file, log_file = scaleml.distributed_train(framework, dist_strategy, train_dataset, log_resources=False)

In [0]:
# def distributed_train(framework, strategy, train_dataset, log_resources=True):
strategy = dist_strategy

import tensorflow as tf
import torch
from datetime import datetime
from multiprocessing import Process, Value
import os, time, pickle

from scaleml import create_scaleml_folders, create_model_tf, create_model_torch, start_logging, stop_logging

# create folders for saving
scaleml_folder = create_scaleml_folders()
print(f"Scaleml folder and subfolders are set up at: {scaleml_folder}")
print()

# log resource use throughout training
log_dir = f"{scaleml_folder}/logs/"
log_file = f"{log_dir}/{datetime.now().strftime('%Y%m%d')}_resource_usage_log.csv"
# if log_resources:        
# Start logging resources in the background (adjust the interval as needed)
# log_process = start_logging(log_file, interval=10)
print()

# Detect size of the first image from train_dataset
train_images = train_dataset.get('images')
train_labels = train_dataset.get('labels')
input_shape = train_images[0].shape

# Save the model
log_time = log_file.split('/')[-1]
log_time = log_time.split('_')[0]

# Start the training process with or without resource monitoring
if framework.get('strategy') == 'tensorflow':
    
    with strategy.scope():
        model = create_model_tf(input_shape, model_path=None)
        
    model.save(os.path.join(f"{scaleml_folder}/models/", f"{log_time}_tf_model.h5"))

    history = model.fit(train_images, train_labels, epochs=2, batch_size=64, verbose=1)
    history_file = os.path.join(f"{scaleml_folder}/models/", f"{log_time}_tf_model_history.pkl")
    # with open(history_file, 'wb') as f:
    #     pickle.dump(history.history, f)

elif framework.get('strategy') == 'pytorch':

    pass

    # history_file = os.path.join(f"{scaleml_folder}/models/", f"{log_time}_torch_model_history.pkl")
    # with open(history_file, 'wb') as f:
    #         pickle.dump(history.history, f)

# if log_resources:
# Stop the logging once training is complete
# stop_logging(log_process)

print("Training completed.")
# return history, history_file, log_file